In [1]:
import textacy
import gensim
import os
import numpy as np
import pandas as pd

In [2]:
review_panel = pd.read_csv('/Users/xiaowan/Downloads/reviews_panel_t8.csv')#'/Users/xiaowan/Documents/SIT Course/word2vec/data/name_reviews_gender_fulltable.csv')
review_panel.head()

,Id,review_id,href,overall_rating,Ease_of_Appointment,Promptness,Courteous_Staff,Accurate_Diagnosis,Bedside_Manner,Spends_Time_with_Me,...,Advanced_Technology,Caring_Manner,Pain_Minimized,Satisfaction,date,reviewer,title,content,helpful_vote,crawl_date
0,12949966,23507540,/dentists/Dr_Aarika_Anderson_Elter,5,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,5.0,NaN,2012-11-16,NaN,Great results,Dr. Anderson explained in detail my options fo...,NaN,2016-09-06 20:54:01
1,12949967,27184431,/dentists/Dr_Aamir_Wahab,5,5.0,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,2015-03-15,Brad s.,implant,Had an implant done and it was painless. I cou...,NaN,2016-09-06 20:54:10
2,12949968,26307282,/dentists/Dr_Aanal_Parikh,1,1.0,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,2014-10-12,Bill johnson,Warning ..would not see this dentist,Do not go to this dentist office they scam peo...,NaN,2016-09-06 20:54:13
3,12949969,28904504,/dentists/Dr_Aaron_Aguilar,5,5.0,NaN,NaN,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,2015-06-11,NaN,NaN,Very thoughtful Dr. Communicates/bedside mann...,NaN,2016-09-06 20:54:14
4,12949970,28380953,/dentists/Dr_Aaron_D_Larsen,4,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,4.0,4.0,5.0,2015-03-31,NaN,Dr larsen great others no,Would continue to see Dr. Larsen however his ...,NaN,2016-09-06 20:54:18


In [3]:
df_review = review_panel.loc[:,['href','content']]
df_review.head()

,href,content
0,/dentists/Dr_Aarika_Anderson_Elter,Dr. Anderson explained in detail my options fo...
1,/dentists/Dr_Aamir_Wahab,Had an implant done and it was painless. I cou...
2,/dentists/Dr_Aanal_Parikh,Do not go to this dentist office they scam peo...
3,/dentists/Dr_Aaron_Aguilar,Very thoughtful Dr. Communicates/bedside mann...
4,/dentists/Dr_Aaron_D_Larsen,Would continue to see Dr. Larsen however his ...


In [4]:
pd.isnull(df_review).sum()

href       0
content    2
dtype: int64

In [5]:
df_review=df_review.dropna()
pd.isnull(df_review).sum()

href       0
content    0
dtype: int64

In [6]:
# Use first 500 reviews to do experiment
sample=df_review.iloc[0:499,1] 
type(sample)

pandas.core.series.Series

In [7]:
sample.to_csv('sample_reviews.csv',index=False,header=False)

## Pre-process via textacy

### Problem: textacy-0.3.4 does not contain __io__ module; cannot transform text file

In [8]:
sample=textacy.io.read_csv('/Users/xiaowan/Documents/SIT Course/word2vec/data/sample_review.csv',
                encoding=None, fieldname=None, dialect='csv')

AttributeError: module 'textacy' has no attribute 'io'

In [9]:
sample.to_csv('sample_reviews.txt', index=False, header=False)

In [10]:
sample_text=open('/Users/xiaowan/Documents/SIT Course/word2vec/sample_reviews.txt').read()

In [11]:
sample_text

'"Dr. Anderson explained in detail my options for the repair on my tooth. In the end, I received great results."\nHad an implant done and it was painless. I couldn\'t afford it so he put me on a payment plan. I\'m pleased with the service. Great doctor\nDo not go to this dentist office they scam people and do not pay there bills.  We snow plowed for them and when the season was over they would not pay us.  This is just to let you know what type of dentist your dealing with.\nVery thoughtful Dr.  Communicates/bedside manners great.\nWould continue to see Dr.  Larsen however his front desk secretary is something else it saddens me to have to give my money to someone else \nDr. Acres and his staff are an exceptionally professional and caring team of health professionals that I highly recommend. \n"After smiling and shaking my hand he was just cold. I was basically ordered where to sit because I was too slow for him I guess. Didn\'t ask me how I was or try to get to know me in the least. I

In [14]:
textacy.preprocess_text(sample_text, lowercase=True, no_punct=False)[:50]

'"dr. anderson explained in detail my options for t'

In [15]:
text_stream, metadata_stream = textacy.io.split_records(sample_text, 'text')
corpus = textacy.Corpus('en', texts=text_stream, metadatas=metadata_stream)

AttributeError: module 'textacy' has no attribute 'io'

## Process via Gensim

In [30]:
sample_list=list(sample)
sample_list

['Dr. Anderson explained in detail my options for the repair on my tooth. In the end, I received great results.',
 "Had an implant done and it was painless. I couldn't afford it so he put me on a payment plan. I'm pleased with the service. Great doctor",
 'Do not go to this dentist office they scam people and do not pay there bills.  We snow plowed for them and when the season was over they would not pay us.  This is just to let you know what type of dentist your dealing with.',
 'Very thoughtful Dr.  Communicates/bedside manners great.',
 'Would continue to see Dr.  Larsen however his front desk secretary is something else it saddens me to have to give my money to someone else ',
 'Dr. Acres and his staff are an exceptionally professional and caring team of health professionals that I highly recommend. ',
 "After smiling and shaking my hand he was just cold. I was basically ordered where to sit because I was too slow for him I guess. Didn't ask me how I was or try to get to know me in

In [33]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in sample.lower().split() if word not in stoplist]
         for sample in sample_list]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)

for text in texts:
     for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)

[['dr.',
  'explained',
  'detail',
  'my',
  'options',
  'repair',
  'on',
  'my',
  'tooth.',
  'i',
  'received',
  'great',
  'results.'],
 ['had',
  'an',
  'implant',
  'done',
  'it',
  'was',
  'painless.',
  'i',
  "couldn't",
  'it',
  'so',
  'he',
  'put',
  'me',
  'on',
  'payment',
  "i'm",
  'pleased',
  'with',
  'service.',
  'great',
  'doctor'],
 ['do',
  'not',
  'go',
  'this',
  'dentist',
  'office',
  'they',
  'people',
  'do',
  'not',
  'pay',
  'there',
  'we',
  'plowed',
  'them',
  'when',
  'was',
  'over',
  'they',
  'would',
  'not',
  'pay',
  'this',
  'is',
  'just',
  'let',
  'you',
  'know',
  'what',
  'type',
  'dentist',
  'your',
  'dealing',
  'with.'],
 ['very', 'dr.', 'manners', 'great.'],
 ['would',
  'continue',
  'see',
  'dr.',
  'however',
  'his',
  'front',
  'desk',
  'is',
  'something',
  'else',
  'it',
  'me',
  'have',
  'give',
  'my',
  'money',
  'someone',
  'else'],
 ['dr.',
  'his',
  'staff',
  'are',
  'an',
  'prof

  'payne',
  'years.',
  'he',
  'is',
  'always',
  'honest',
  'will',
  'do',
  'best',
  'you.',
  'he',
  'is',
  'only',
  'dentist',
  'that',
  'i',
  'have',
  'absolutely',
  'not',
  'had',
  'any',
  'pain',
  'from',
  'shots',
  'has',
  'always',
  'been',
  'courteous.',
  'he',
  'does',
  'fantastic',
  'work,',
  'what',
  'more',
  'can',
  'i',
  '.'],
 ['i',
  'had',
  'an',
  'emergency',
  'dental',
  'regular',
  'dentist',
  "didn't",
  'even',
  'return',
  'my',
  'call(',
  'it',
  'was',
  'he',
  'was',
  'supposed',
  'be',
  'on',
  'i',
  'saw',
  'magazine',
  'dr',
  'within',
  '10',
  'minutes',
  'he',
  'my',
  'call(',
  'at',
  'he',
  'had',
  'me',
  'come',
  'next',
  'morning',
  'on',
  'he',
  'me',
  'up',
  'gave',
  'me',
  'temporary',
  'until',
  'i',
  'could',
  'get',
  'back',
  'root',
  'canal.',
  'needless',
  'say,',
  'i',
  'will',
  'be',
  'new',
  'client'],
 ['office',
  'is',
  'clean',
  'not',
  'bad.',
  'but',
 

  'all',
  'digital',
  'things',
  'staff',
  'were',
  'very',
  'friendly.',
  'i',
  'went',
  'have',
  '6',
  'implants',
  'done,',
  'i',
  'was',
  'very',
  'nervous,',
  'but',
  'they',
  'atmosphere',
  'it',
  'really',
  'was.',
  'my',
  'treatment',
  'was',
  'great',
  'when',
  'i',
  'up',
  'i',
  'that',
  'was',
  'best',
  'i',
  'really',
  'appreciated',
  'professional',
  'demeanor',
  'his',
  'staff',
  'as',
  'well.'],
 ['my',
  'mother',
  'had',
  'most',
  'difficult',
  'time',
  'using',
  'her',
  'dentures',
  'she',
  'is',
  'after',
  'suffering',
  'many',
  'problems',
  'with',
  'dr',
  'shakfa',
  'his',
  'nurses',
  'were',
  'best.',
  'my',
  'year',
  'old',
  'mother',
  'was',
  'very',
  'she',
  'was',
  'was',
  'given',
  '8',
  'implants',
  'new',
  'teeth',
  'chew',
  'with',
  'no',
  'time',
  'she',
  'having',
  'best',
  'ever.',
  'watching',
  'her',
  'eat',
  'again',
  'makes',
  'me',
  'so',
  'watching',
  'my'

  'caring',
  'very',
  'polite.',
  'i',
  'will',
  'go',
  'back',
  'very',
  'happy'],
 ['dr',
  'is',
  'not',
  'only',
  'professional',
  'his',
  'abilities',
  'demeanor',
  'trust'],
 ["it's", 'good', 'see', 'friendly', 'staff', 'that', 'cares'],
 ['i',
  'was',
  'grateful',
  'that',
  'dr.',
  'adam',
  'took',
  'time',
  'get',
  'me',
  'in.',
  'i',
  'had',
  'jaw',
  'tooth',
  'that',
  'had',
  'lost',
  'its',
  'filling',
  'about',
  'tooth',
  'had',
  'broke',
  'off.',
  'i',
  'had',
  'pain',
  'my',
  'jaw',
  'bone',
  'down',
  'my',
  'neck',
  'which',
  'concerned',
  'me,',
  'as',
  'i',
  'had',
  'heard',
  'horror',
  'stories',
  'about',
  'jaw',
  'bone',
  'he',
  'was',
  'most',
  'polite,',
  'informative,',
  'compassionate,',
  'i',
  "can't",
  'imagine',
  'how',
  'things',
  'would',
  'have',
  'turned',
  'out',
  'if',
  'he',
  "couldn't",
  'see',
  'me.',
  'he',
  'had',
  'great',
  'sense',
  'humor',
  'his',
  'assistant

  "he's",
  'not',
  'you',
  'all',
  'should',
  'totally',
  'check',
  'out',
  'adam.',
  "he's",
  'got',
  'set',
  'skills',
  'that',
  'could',
  'turn',
  'any',
  'straight',
  'man',
  'y'],
 ['i',
  'saw',
  'dr.',
  's.',
  'mcdonough',
  'ga',
  'dentist',
  'office',
  'it',
  'was',
  'worst',
  'experience',
  'ever',
  'from',
  'any',
  'dentist',
  'i',
  'have',
  'had',
  'dental',
  'work',
  'done',
  'by.',
  'read',
  'detail',
  'how',
  'he',
  'damaged',
  'me',
  'i',
  "don't",
  'understand',
  'it',
  'is',
  'detailed',
  'on',
  'other',
  'reviews',
  'mcdonough',
  'ga',
  'dentistry',
  'office',
  'reviews.',
  'i',
  'am',
  'still',
  'man',
  'as',
  'if',
  'he',
  'put',
  'me',
  'pain,',
  'it',
  'has',
  'been',
  'two',
  'now.',
  'i',
  'am',
  'still',
  'still',
  'still',
  ',',
  ',',
  'from',
  'top',
  'eye',
  'bottom',
  'very',
  'unprofessional.',
  'his',
  'young',
  'assistance',
  'told',
  'me',
  'that',
  'my',
  'h

  'appointment',
  'week',
  'later.',
  'like',
  'i',
  'i',
  'have',
  'never',
  'had',
  'any',
  'issues',
  'with',
  'pain',
  'or',
  'anxiety.',
  'after',
  'he',
  'started',
  'he',
  'kept',
  'yelling',
  'at',
  'me',
  'open',
  'wider.',
  'i',
  'was',
  'so',
  'wide',
  'that',
  'my',
  'were',
  'it',
  'was',
  'not',
  'good',
  'enough',
  'him.',
  'he',
  'got',
  'left',
  'came',
  'back',
  'few',
  'minutes',
  'later',
  'finished',
  'which',
  'him',
  'yelling',
  'at',
  'me',
  'two',
  'more',
  'when',
  'he',
  'was',
  'done,',
  'he',
  'stood',
  'up',
  'walked',
  'out',
  'without',
  'saying',
  'anything',
  'me.',
  'his',
  'assistant',
  'told',
  'me',
  'have',
  'nice',
  'day',
  'that',
  'was',
  'have',
  'never',
  'my',
  'life',
  'been',
  'treated',
  'like',
  'that',
  "doctor's",
  'office',
  'before.',
  'i',
  'will',
  'never',
  'go',
  'back',
  'there',
  'away',
  'from',
  'this',
  'doctor!'],
 ['i',
  'can',

  'doctor',
  'that',
  'would',
  'sue',
  'one',
  'his',
  'patients',
  'on'],
 ['this',
  'is',
  'best',
  'that',
  'i',
  'have',
  'ever',
  'been',
  'to.',
  'i',
  'saw',
  'him',
  'when',
  'he',
  'was',
  'now',
  'that',
  'e',
  'moved',
  "he's",
  'very',
  'questions',
  'takes',
  'his',
  'time',
  'with',
  'you,',
  'very',
  'honest',
  'must',
  'has',
  'hands',
  "don't",
  'feel',
  'last',
  'time',
  'i',
  'went',
  'see',
  'him,',
  'as',
  'i',
  'was',
  'out',
  'office',
  'i',
  'my',
  'husband',
  'who',
  'was',
  'waiting',
  'room,',
  'how',
  'good',
  'doctor',
  'had',
  'been',
  'another',
  'patient',
  'there',
  'told',
  'me',
  'so',
  'good',
  'that',
  'he',
  'should',
  'be',
  'giving',
  'other'],
 ['staff', 'doctor.', 'never', 'have', 'wait'],
 ['off',
  'ever!',
  "i've",
  'never',
  'had',
  'such',
  'an',
  'awful',
  'experience',
  'with',
  'any',
  'doctor!',
  "he's",
  'it',
  'money',
  'awful',
  'bedside',
  

  'he',
  'refused',
  'correct',
  'bad',
  'job',
  'done',
  'even',
  'at',
  'an',
  'extra',
  'no',
  'i',
  'am',
  'with',
  'tooth',
  'until',
  'i',
  'am',
  'able',
  'get',
  'another',
  'doc',
  'job.',
  'on',
  'minor',
  "he's",
  'known',
  'cancel',
  '&',
  'reschedule',
  'with',
  'short',
  'which',
  'at',
  'times',
  'made',
  'my',
  'schedule'],
 ['i',
  'had',
  'been',
  'scared',
  'dentist',
  'am',
  'years',
  'old',
  'had',
  'found',
  'dr',
  'calderone',
  '2',
  'years',
  'ago',
  'is',
  'best.',
  'he',
  'takes',
  'time',
  'explain',
  'things',
  'wants',
  'you',
  'pain',
  'free.',
  'i',
  'had',
  'problem',
  'one',
  'he',
  'called',
  'me',
  'back',
  'right',
  'someone',
  'told',
  'me',
  'there',
  'was',
  'better',
  'dentist',
  'i',
  'believe',
  'them.',
  'doctor',
  'calderone',
  'is',
  'best'],
 ['i',
  'have',
  'been',
  'patient',
  'dental',
  '15',
  'years.',
  'before',
  'dr.',
  'calderone',
  'took',


  'dr.',
  'winton.',
  'plus,',
  'both',
  'his',
  'offices',
  'are',
  'spotless',
  'clean',
  'state',
  "he's",
  'man!'],
 ['highly', 'recommend', 'this', 'oral', 'surgeon.'],
 ['simply',
  'great',
  'oral',
  'surgeon',
  '-',
  'skilled,',
  'informative,',
  'no',
  'pain,',
  'filled',
  'me',
  'with',
  'confidence',
  'his',
  'i',
  'would',
  'recommend',
  'him',
  'anyone.'],
 ['i',
  'am',
  'always',
  'terrified',
  'dentists,',
  'but',
  'dr.',
  'winton',
  'his',
  'staff',
  'ease',
  'your',
  'fears',
  'prepare',
  'you',
  'quality',
  'time',
  'with',
  'you,',
  'kind',
  'bedside',
  'manner',
  'find',
  'detail',
  'your',
  'next',
  'calls',
  'day',
  'after',
  'procedure',
  'check',
  'on',
  'available',
  '24',
  'after',
  'procedure'],
 ['just',
  'had',
  'wonderful',
  'experience',
  'with',
  'who',
  'is',
  'my',
  'hygienist,',
  'very',
  'professional.',
  'also',
  'want',
  'who',
  'always',
  'is',
  'very',
  'helpful',
  '

In [36]:
# To convert documents to vectors
from gensim import corpora
dictionary = corpora.Dictionary(texts)
#dictionary.save('/tmp/try_dictionary.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(1399 unique tokens: ['detail', 'dr.', 'explained', 'great', 'i']...)


In [37]:
# To see the mapping between words and their ids
print(dictionary.token2id)

{'detail': 0, 'dr.': 1, 'explained': 2, 'great': 3, 'i': 4, 'my': 5, 'on': 6, 'options': 7, 'received': 8, 'repair': 9, 'results.': 10, 'tooth.': 11, 'an': 12, "couldn't": 13, 'doctor': 14, 'done': 15, 'had': 16, 'he': 17, "i'm": 18, 'implant': 19, 'it': 20, 'me': 21, 'painless.': 22, 'payment': 23, 'pleased': 24, 'put': 25, 'service.': 26, 'so': 27, 'was': 28, 'with': 29, 'dealing': 30, 'dentist': 31, 'do': 32, 'go': 33, 'is': 34, 'just': 35, 'know': 36, 'let': 37, 'not': 38, 'office': 39, 'over': 40, 'pay': 41, 'people': 42, 'plowed': 43, 'them': 44, 'there': 45, 'they': 46, 'this': 47, 'type': 48, 'we': 49, 'what': 50, 'when': 51, 'with.': 52, 'would': 53, 'you': 54, 'your': 55, 'great.': 56, 'manners': 57, 'very': 58, 'continue': 59, 'desk': 60, 'else': 61, 'front': 62, 'give': 63, 'have': 64, 'his': 65, 'however': 66, 'money': 67, 'see': 68, 'someone': 69, 'something': 70, 'are': 71, 'caring': 72, 'health': 73, 'highly': 74, 'professional': 75, 'professionals': 76, 'recommend.': 7

In [34]:
# train word2vec on the texts
model = gensim.models.Word2Vec(texts, min_count=1)

# to save or load a trained model
#model.save(fname)
#model = Word2Vec.load(fname)

In [38]:
model.wv['doctor']  # numpy vector of a word

array([ 0.07998582, -0.32902834,  0.04222899, -0.0287748 , -0.25997457,
       -0.07039756, -0.15080042,  0.14474258, -0.11551479, -0.17456922,
       -0.06993326,  0.31485584,  0.22920072, -0.08220432, -0.14026117,
        0.13937953,  0.04824394,  0.05291249,  0.09243712,  0.09108059,
        0.06209139, -0.07202045, -0.13594411,  0.24247845, -0.32266366,
        0.11776014,  0.03025448, -0.1678866 ,  0.16025653,  0.03385702,
        0.14359836,  0.23757879,  0.17445023,  0.26062134,  0.06891587,
        0.31761503,  0.10824645, -0.15772727, -0.15886028,  0.09612776,
        0.2419533 , -0.0222924 , -0.14265928, -0.44701502, -0.19945391,
       -0.55472308,  0.02628469, -0.23427127, -0.16521183, -0.17634416,
        0.08375294, -0.13288788, -0.27756032,  0.10010162,  0.13653859,
        0.15990651,  0.07371777,  0.17631318,  0.15064313,  0.06414368,
       -0.40808389,  0.12224804,  0.20780283,  0.27148047, -0.05474283,
        0.1552041 , -0.00954272,  0.12185957,  0.07405337,  0.19

In [40]:
# to find similar words
model.wv.most_similar(positive=['she', 'professional'], negative=['he'])

[('very', 0.9997639656066895),
 ('about', 0.9997638463973999),
 ('was', 0.9997581243515015),
 ('it', 0.9997569918632507),
 ('i', 0.9997550845146179),
 ('his', 0.9997532963752747),
 ('they', 0.9997528791427612),
 ('dr.', 0.9997521638870239),
 ('not', 0.9997517466545105),
 ('when', 0.9997491836547852)]

In [41]:
model.wv.most_similar(positive=['she', 'nice'], negative=['he'])

[('as', 0.999634861946106),
 ('it', 0.9996311664581299),
 ('out', 0.9996266961097717),
 ('i', 0.9996243715286255),
 ('professional', 0.9996219873428345),
 ('they', 0.9996200799942017),
 ('but', 0.9996193647384644),
 ('was', 0.9996165633201599),
 ('dr.', 0.9996152520179749),
 ('not', 0.9996140003204346)]